In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

def calculateFeature(filename_to_read,filename_to_write): 
    # calculate the feature from filename_to_read (csv) 
    # write the feature to filename_to_write (csv)
    # return a dataFrame that contains the feature
    
    df = pd.read_csv(filename_to_read)
    
    MouseA_ax = df[('mouseA_1_X')]
    MouseA_ay = df[('mouseA_1_Y')]
    MouseB_ax = df[('mouseB_1_X')]
    MouseB_ay = df[('mouseB_1_Y')]

    MouseA_cx = df[('mouseA_3_X')]
    MouseA_cy = df[('mouseA_3_Y')]
    MouseB_cx = df[('mouseB_3_X')]
    MouseB_cy = df[('mouseB_3_Y')]

    MouseA_ex = df[('mouseA_5_X')]
    MouseA_ey = df[('mouseA_5_Y')]
    MouseB_ex = df[('mouseB_5_X')]
    MouseB_ey = df[('mouseB_5_Y')]
 
    # previous: drop the last element; after: drop the first element
    # easier to perform calculation
    previous_Ax = MouseA_cx.drop(index=MouseA_cx.index[-1], axis=0, inplace=False)
    after_Ax = MouseA_cx.drop(index=MouseA_cx.index[0], axis=0, inplace=False)
    previous_Ay = MouseA_cy.drop(index=MouseA_cy.index[-1], axis=0, inplace=False)
    after_Ay = MouseA_cy.drop(index=MouseA_cy.index[0], axis=0, inplace=False) 
  
    previous_Bx = MouseB_cx.drop(index=MouseB_cx.index[-1], axis=0, inplace=False)
    after_Bx = MouseB_cx.drop(index=MouseB_cx.index[0], axis=0, inplace=False)
    previous_By = MouseB_cy.drop(index=MouseB_cy.index[-1], axis=0, inplace=False)
    after_By = MouseB_cy.drop(index=MouseB_cy.index[0], axis=0, inplace=False) 

    # reset the index to keep consistency
    previous_Ax.reset_index(drop=True,inplace=True)
    after_Ax.reset_index(drop=True,inplace=True)
    previous_Ay.reset_index(drop=True,inplace=True)
    after_Ay.reset_index(drop=True,inplace=True)

    previous_Bx.reset_index(drop=True,inplace=True)
    after_Bx.reset_index(drop=True,inplace=True)
    previous_By.reset_index(drop=True,inplace=True)
    after_By.reset_index(drop=True,inplace=True)
    
    # prepare for calculating angle
    mouseA_dir_dis = np.sqrt(pow((MouseA_ax-MouseA_cx),2) + pow((MouseA_ay - MouseA_cy),2))
    mouseA_to_mouseB = np.sqrt(pow((MouseA_ax-MouseB_cx),2) + pow((MouseA_ay - MouseB_cy),2))
    mouseB_dir_dis = np.sqrt(pow((MouseB_ax-MouseB_cx),2) + pow((MouseB_ay - MouseB_cy),2))
    mouseB_to_mouseA = np.sqrt(pow((MouseB_ax-MouseA_cx),2) + pow((MouseB_ay - MouseA_cy),2))
    
    # calculation
    nose_to_nose = np.sqrt(pow((MouseA_ax-MouseB_ax),2) + pow((MouseA_ay - MouseB_ay),2))
    noseA_to_rearB = np.sqrt(pow((MouseA_ax-MouseB_ex),2) + pow((MouseA_ay - MouseB_ey),2))
    noseB_to_rearA = np.sqrt(pow((MouseB_ax-MouseA_ex),2) + pow((MouseB_ay - MouseA_ey),2))
    proximity = np.sqrt(pow((MouseA_cx-MouseB_cx),2) + pow((MouseA_cy - MouseB_cy),2))
    velocity_A = np.sqrt(pow((previous_Ax-after_Ax),2) + pow((previous_Ay - after_Ay),2))
    velocity_B = np.sqrt(pow((previous_Bx-after_Bx),2) + pow((previous_By - after_By),2))
    angle_AtoB = np.arccos((pow(mouseA_dir_dis,2) + pow(mouseA_to_mouseB,2) - pow(proximity,2))/(2*mouseA_dir_dis*mouseA_to_mouseB))
    angle_BtoA = np.arccos((pow(mouseB_dir_dis,2) + pow(mouseB_to_mouseA,2) - pow(proximity,2))/(2*mouseB_dir_dis*mouseB_to_mouseA))
    
    # drop the last element to make all the data to be the same length 
    # velocity has one element less than others
    nose_to_nose.drop(index=nose_to_nose.index[-1], axis=0, inplace=True)
    noseA_to_rearB.drop(index=noseA_to_rearB.index[-1], axis=0, inplace=True)
    noseB_to_rearA.drop(index=noseB_to_rearA.index[-1], axis=0, inplace=True)
    proximity.drop(index=proximity.index[-1], axis=0, inplace=True)
    angle_AtoB.drop(index=angle_AtoB.index[-1], axis=0, inplace=True)
    angle_BtoA.drop(index=angle_BtoA.index[-1], axis=0, inplace=True)
    speed_diff = np.absolute(velocity_A-velocity_B)
    
    # write data
    data = {'nose_to_nose':nose_to_nose, 'noseA_to_rearB':noseA_to_rearB, 'noseB_to_rearA':noseB_to_rearA,'proximity':proximity, 'velocity_A':velocity_A,'velocity_B':velocity_B,'angle_AtoB':angle_AtoB,'angle_BtoA':angle_BtoA,'speed_diff':speed_diff}
    data = pd.DataFrame(data)
    data.dropna().to_csv(filename_to_write)

    return data

In [ ]:
# specify the directory path
directory = "/path/to/directory"

# iterate over each file in the directory
for filename in os.listdir(directory):
    # check if the file is a regular file (i.e. not a directory)
    if os.path.isfile(os.path.join(directory, filename)):
        # print the file name
        print(filename)
        calculateFeature(filename,"Feature_" + filename)

In [ ]:
# # iterate through different datafile
# for i in range(n):   
#     num = '' + 1
#     file_to_read = 'XXX' + num + 'XXX.csv'
#     file_to_write = 'XXX' + num + 'XXX.csv'
#     calculateFeature(file_to_read,file_to_write)